In [3]:
import trimesh

scene = trimesh.scene.Scene()

In [ ]:
# load file /home/s94zalek/transmatching/evaluation/datasets/shrec19/data/000/A.off

scene.geometry.clear()

mesh = trimesh.load_mesh('/home/s94zalek/transmatching/evaluation/datasets/shrec19/data/000/A.off')
scene.add_geometry(mesh)

# load file /home/s94zalek/transmatching/evaluation/datasets/shrec19/data/000/B.off

mesh = trimesh.load_mesh('/home/s94zalek/transmatching/evaluation/datasets/shrec19/data/000/B.off')
mesh.vertices += [1, 0, 0]
scene.add_geometry(mesh)

scene.show()

In [ ]:
'/home/s94zalek/transmatching/evaluation/datasets/shrec19/origin_shrec/mat/1.ply'

scene.geometry.clear()

mesh = trimesh.load_mesh('/home/s94zalek/transmatching/evaluation/datasets/shrec19/origin_shrec/mat/1.ply')
scene.add_geometry(mesh)

# load file /home/s94zalek/transmatching/evaluation/datasets/shrec19/origin_shrec/mat/2.ply

mesh = trimesh.load_mesh('/home/s94zalek/transmatching/evaluation/datasets/shrec19/origin_shrec/mat/2.ply')
mesh.vertices += [1, 0, 0]
scene.add_geometry(mesh)

scene.show()

In [ ]:
# read /home/s94zalek/transmatching/evaluation/datasets/shrec19/origin_shrec/mat/2.mat

import scipy.io

mat = scipy.io.loadmat('/home/s94zalek/transmatching/evaluation/datasets/shrec19/origin_shrec/mat/1.mat')

# for key in mat:
#     print(f'### {key}: {mat[key]}')

print(mat.keys())
print(mat['smpl_matches'].shape)

In [ ]:
# read /home/s94zalek/transmatching/evaluation/datasets/shrec19/SHREC19_matching_humans/mat/1.mat

mat_inside = scipy.io.loadmat('/home/s94zalek/transmatching/evaluation/datasets/shrec19/SHREC19_matching_humans/mat/1.mat')

print(mat_inside.keys())
print(mat_inside['smpl_matches'].shape)

In [ ]:
# read /home/s94zalek/transmatching/evaluation/datasets/faust_1k_s2t/12ktemplate.ply

scene.geometry.clear()

mesh = trimesh.load_mesh('/home/s94zalek/transmatching/evaluation/datasets/faust_1k_s2t/12ktemplate.ply')
scene.add_geometry(mesh)

scene.show()

In [ ]:
# read /home/s94zalek/transmatching/evaluation/datasets/faust_s2t/1k_to_faust_matchings.mat

mat = scipy.io.loadmat('/home/s94zalek/transmatching/evaluation/datasets/faust_s2t/1k_to_faust_matchings.mat')

for key in mat:
    print(f'### {key}')

mat['idx_rem'].shape

In [ ]:
# read /home/s94zalek/transmatching/evaluation/datasets/faust_1k_noise/FAUST_noise_0.01.mat

mat = scipy.io.loadmat('/home/s94zalek/transmatching/evaluation/datasets/faust_1k_noise/FAUST_noise_0.01.mat')

# print(mat.keys())
# for key in mat:
#     print(f'### {key}: {mat[key]}')
# mat['vertices_clean'].shape

In [ ]:
# read /home/s94zalek/shape_matching/data/D-FAUST/registrations_m.hdf5

import h5py

f = h5py.File('/home/s94zalek/shape_matching/data/D-FAUST/registrations_m.hdf5', 'r')

# for key in f:
#     print(f'### {key}: {f[key]}')

# print(f['50002_chicken_wings'][:, :, 0].shape)

scene.geometry.clear()

mesh = trimesh.Trimesh(vertices=f['50002_chicken_wings'][:, :, 90], faces=f['faces'])

scene.add_geometry(mesh)

scene.show()

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm

faust_base = '/home/s94zalek/transmatching/evaluation/datasets/faust_s2t/data'
# faust_original = '/home/s94zalek/transmatching/evaluation/datasets/faust_original'
faust_original = '/home/s94zalek/shape_matching/data/FAUST_original/off'
faust_original_corres = '/home/s94zalek/shape_matching/data/FAUST_original/corres'
os.makedirs(faust_original, exist_ok=True)
os.makedirs(faust_original_corres, exist_ok=True)

faust_dict = {}

n_verts = 6890
n_faces = 13776

for folder in tqdm(os.listdir(faust_base), desc='Loading FAUST'):
    with open(f'{faust_base}/{folder}/meta.json', 'r') as f:
        meta = json.load(f)
    
    mesh_b = trimesh.load_mesh(f'{faust_base}/{folder}/B.off', process=False)
    assert mesh_b.vertices.shape[0] == n_verts
    assert mesh_b.faces.shape[0] == n_faces
    
    faust_dict[meta['id']['B']] = mesh_b
    

corres = np.array([i for i in range(n_verts)]) + 1
for mesh_name in tqdm(sorted(faust_dict.keys()), desc='Saving FAUST original'):
    # save mesh to faust_original
    mesh = faust_dict[mesh_name]
    mesh.export(f'{faust_original}/{mesh_name[:-4]}.off')
    
    
    with open(f'{faust_original_corres}/{mesh_name[:-4]}.vts', 'w') as f:
        np.savetxt(f, corres, fmt='%d', delimiter='\n')

In [ ]:
shrec_base = '/home/s94zalek/transmatching/evaluation/datasets/shrec19/SHREC19_matching_humans/mat'
shrec_original_off = '/home/s94zalek/shape_matching/data/SHREC19_original/off'
shrec_original_corres = '/home/s94zalek/shape_matching/data/SHREC19_original/corres'

os.makedirs(shrec_original_off, exist_ok=True)
os.makedirs(shrec_original_corres, exist_ok=True)


for file in tqdm(os.listdir(shrec_base), desc='Loading SHREC19'):
    mat = scipy.io.loadmat(f'{shrec_base}/{file}')
    
    # shape_df = mat['Shape_df']
    name = mat['name']
    smpl_matches = mat['smpl_matches']
    
    vert = mat['Shape_df'][0][0][0]
    triv = mat['Shape_df'][0][0][1]
    
    mesh = trimesh.Trimesh(vertices=vert, faces=triv, process=False)
    
    assert mesh.vertices.shape[0] == vert.shape[0]
    assert mesh.faces.shape[0] == triv.shape[0]
    
    mesh.export(f'{shrec_original_off}/{file[:-4]}.off')
    
    with open(f'{shrec_original_corres}/{file[:-4]}.vts', 'w') as f:
        np.savetxt(f, smpl_matches, fmt='%d', delimiter='\n')
    
    
    


# mat_inside = scipy.io.loadmat('/home/s94zalek/transmatching/evaluation/datasets/shrec19/SHREC19_matching_humans/mat/1.mat')

# print(mat_inside.keys())
# print('Shape_df', 'name')
# print(mat_inside['name'])

# vert = mat_inside['Shape_df'][0][0][0]
# triv = mat_inside['Shape_df'][0][0][1]
# n_verts = mat_inside['Shape_df'][0][0][2]
# n_faces = mat_inside['Shape_df'][0][0][3]

# # print(mat_inside['Shape_df'][0][0][0])

# print('vert', vert.shape)
# print('triv', triv.shape)
# print('n_verts', n_verts)
# print('n_faces', n_faces)
# print(mat_inside['smpl_matches'].shape, mat_inside['smpl_matches'].min(), mat_inside['smpl_matches'].max())

In [10]:
# read /home/s94zalek/shape_matching/data/SURREAL_full/full_datasets/dataset_158_158_316_0_32_93/test/second/poses.npy
import numpy as np

poses = np.load('/home/s94zalek/shape_matching/data/SURREAL_full/full_datasets/dataset_158_158_316_0_32_93/test/second/poses.npy')
betas = np.load('/home/s94zalek/shape_matching/data/SURREAL_full/full_datasets/dataset_158_158_316_0_32_93/test/second/betas.npy')

verts = np.load('/home/s94zalek/shape_matching/data/SURREAL_full/full_datasets/dataset_158_158_316_0_32_93/test/second/verts.npy')
faces = np.load('/home/s94zalek/shape_matching/data/SURREAL_full/full_datasets/dataset_158_158_316_0_32_93/test/second/faces.npy')

print('poses.shape', poses.shape, 'betas.shape', betas.shape)

poses.shape (6990, 72) betas.shape (6990, 10)


In [12]:
pose_0[15]

array([ 0.27407107,  0.04282999, -0.11409264], dtype=float32)

In [23]:
import sys
sys.path.append('/home/s94zalek/shape_matching/my_code/datasets/')

from smpl_webuser.serialization import load_model


m_male = load_model("/home/s94zalek/shape_matching/data/SURREAL_full/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl")

In [38]:

## Assign gaussian pose
m_male.pose[:] = poses[0]
m_male.betas[:] = betas[0]
# m.pose[0:3]=0
    
point_set = m_male.r.astype(np.float32)
joints = m_male.J_transformed.r.astype(np.float32)

#normalize
centroid = np.expand_dims(np.mean(point_set[:,0:3], axis = 0), 0) #Useless because dataset has been normalised already
point_set[:,0:3] = point_set[:,0:3] - centroid
joints[:,0:3] = joints[:,0:3] - centroid

# print(dir(m_male))
# print(m_male.J, m_male.J_transformed)

# print(point_set.shape, point_set)

In [ ]:
# plot joints and point_set in 3D

scene.geometry.clear()

pcl_joints = trimesh.points.PointCloud(joints, colors=[255, 0, 0], size=100)
pcl_verts = trimesh.points.PointCloud(point_set, colors=[0, 0, 255], size=100)

scene.add_geometry(pcl_joints)
scene.add_geometry(pcl_verts)

scene.show()

In [ ]:
pr

In [ ]:


# rearrange the coordinate axes
# pose_0 = pose_0[:, [1, 2, 0]]

# plot 3D points

scene.geometry.clear()

pcl = trimesh.PointCloud(vertices=verts[0][:24], colors=[0, 0, 255], size=100)
mesh = trimesh.Trimesh(vertices=verts[0], faces=faces[0])

# scene.add_geometry(mesh)
scene.add_geometry(pcl)


scene.show()